In [1]:
import sys
import numpy as np
from IPython.display import display
from maraboupy import Marabou, MarabouCore
from utils import TOTUtils

In [3]:
nnet_path = '../artifacts/models/model_v3.2.0/model.h5'
data_csv = '../data/v3.2.0/test.csv'
all_samples = TOTUtils.load_samples(data_csv, frac=0.1)
samples = TOTUtils.filter_samples(all_samples, nnet_path)
print(f'{len(samples)} correct samples')

13528 correct samples


In [3]:
from robustness import find_local_robustness_boundaries

h5_path = '../artifacts/models/model_v3.2.0/model.h5'
results = find_local_robustness_boundaries(h5_path, samples, d_min=0.00001, d_max=100, multithread=True, save_results=True, save_samples=True)
summary, _ = results
print(f'summary: {summary}')

INFO:robustness:wrote summary to ../logs/robustness/local_summary.csv
INFO:robustness:wrote detils to ../logs/robustness/local_details.csv
summary: (-0.0047000000000000635, 0.007550000000000065)
